In [46]:
# ---- Cleanup everything before start ----
rm(list = ls())
gc()

### Find true capital-state pairs from all possible capital-state pairs

# ---- GBSERVER API ----
source("./experimentAPI.R")

# ---- INPUT and CONFIGURATIONS ----

EDGE_TYPE_FILE = "../data/infobox.edgetypes" # Example : "../data/lobbyist.edgetypes"
INPUT_FILE = "/home/phi/KGMiner/facts/state_capital.csv" # Example : "../facts/lobbyist/firm_payee.csv" col 1 and 2 are ids and 3 is label
CLUSTER_SIZE = 48 # Number of workers in gbserver
FALSE_PER_TRUE = 5
DISCARD_REL = 191
ASSOCIATE_REL = c(404)
max_depth = 3


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604868,32.4,1168576,62.5,1168576,62.5
Vcells,1095473,8.4,1919474,14.7,1919474,14.7


In [2]:
options(warn=-1)

In [18]:
mapfile <- read.csv(EDGE_TYPE_FILE, sep="\t", header=F)
mapfile$V1 <- as.numeric(mapfile$V1)
mapfile$V2 <- as.character(mapfile$V2)


In [47]:
# ---- Load input data ----
dat.true <- read.csv(INPUT_FILE)

if (ncol(dat.true) < 3)
  dat.true$label <- T

# ---- Construct false labeled data -----
set.seed(233)

# TODO: reformat this so it is universal and file independent
dat.false <- rbind.fill(apply(dat.true, 1, function(x){
  candidates <- unique(dat.true[which(dat.true[,1] != x[1]), 2])
  candidates <- unlist(lapply(candidates, function(y){
    if(length(which(dat.true[,1] == x[1] & dat.true[,2] == y) != 0)) {
      return(NULL)
    }
    return(y)
  }))
  return(data.frame(src=x[1], 
                    dst=sample(candidates, FALSE_PER_TRUE),
                    label=F))
}))

colnames(dat.true) <- c("src","dst","label")
dat <- rbind(dat.true, dat.false)

elapsed.time <- data.frame()

In [63]:
install.packages("readr") 

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [79]:
library(readr)
dat <- read_file("/home/phi/KGMiner/input_R.txt")
dat[1]
dat[-1]

[1] "29,-29,TRUE,\n671,-671,TRUE,\n671,-671,TRUE,\n671,-671,TRUE,\n-40,39,TRUE,\n-38,85,TRUE,\n-38,38,TRUE,\n-115,16,TRUE,\n-40,404,TRUE,\n-40,606,TRUE,\n-40,-44,TRUE,\n-38,46,TRUE,\n-46,46,TRUE,\n-40,39,TRUE,\n-85,85,TRUE,\n-17,17,TRUE,\n-40,39,TRUE,\n-17,17,TRUE,\n-46,46,TRUE,\n-39,240,TRUE,\n-38,103,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-29,253,TRUE,\n-40,72,TRUE,\n-40,73,TRUE,\n-115,16,TRUE,\n-15,16,TRUE,\n-15,17,TRUE,\n-38,38,TRUE,\n-38,46,TRUE,\n-46,46,TRUE,\n-17,17,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-38,38,TRUE,\n-46,46,TRUE,\n-38,38,TRUE,\n-38,46,TRUE,\n-46,38,TRUE,\n-46,46,TRUE,\n-65,38,TRUE,\n-65,46,TRUE,\n-46,46,TRUE,\n-38,38,TRUE,\n-38,103,TRUE,\n-103,103,TRUE,\n-38,38,TRUE,\n-38,103,TRUE,\n-103,38,TRUE,\n-103,103,TRUE,\n-167,46,TRUE,\n-38,103,TRUE,\n-39,240,TRUE,\n-40,39,TRUE,\n-115,157,TRUE,\n-38,46,TRUE,\n-38,38,TRUE,\n-40,39,TRUE,\n-46,38,TRUE,\n-46,167,TRUE,\n-115,16,TRUE,\n-46,38,TRUE,\n-46,167,TRUE,\n-40,39,TRUE,\n-39,240,TRUE,\n-40,39,TRUE,\n-46,46,TRUE,\n-40,39,TRUE,\n-38,38,TRUE,\n-29,253,TRUE,\n-40,39,TRUE,\n-46,38,TRUE,\n-40,39,TRUE,\n-39,39,TRUE,\n-39,39,TRUE,\n-39,240,TRUE,\n-15,16,TRUE,\n-15,17,TRUE,\n-115,16,TRUE,\n-29,253,TRUE,\n-17,17,TRUE,\n-38,46,TRUE,\n-46,46,TRUE,\n-17,17,TRUE,\n-17,17,TRUE,\n-85,85,TRUE,\n-39,240,TRUE,\n-39,240,TRUE,\n-38,38,TRUE,\n-85,85,TRUE,\n-38,38,TRUE,\n-38,38,TRUE,\n-46,46,TRUE,\n-38,103,TRUE,\n-38,38,TRUE,\n-39,240,TRUE,\n-39,240,TRUE,\n-39,39,TRUE,\n-226,226,TRUE,\n-38,38,TRUE,\n-253,253,TRUE,\n-16,16,TRUE,\n-103,103,TRUE,\n-38,38,TRUE,\n-39,39,TRUE,\n-39,39,TRUE,\n-157,157,TRUE,\n-17,17,TRUE,\n-294,294,TRUE,\n-46,46,TRUE,\n-65,46,TRUE,\n-15,16,TRUE,\n-15,17,TRUE,\n-38,38,TRUE,\n-38,103,TRUE,\n-103,38,TRUE,\n-103,103,TRUE,\n-294,294,TRUE,\n-40,39,TRUE,\n-38,38,TRUE,\n-40,39,TRUE,\n-277,38,TRUE,\n-277,277,TRUE,\n-40,39,TRUE,\n-15,16,TRUE,\n-15,17,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-38,46,TRUE,\n-46,46,TRUE,\n-38,38,TRUE,\n-38,46,TRUE,\n-46,38,TRUE,\n-46,46,TRUE,\n-115,16,TRUE,\n29,-29,TRUE,\n671,-671,TRUE,\n671,-671,TRUE,\n671,-671,TRUE,\n-38,38,TRUE,\n-38,38,TRUE,\n-16,16,TRUE,\n-16,17,TRUE,\n-17,16,TRUE,\n-17,17,TRUE,\n-44,TRUE,\n-38,38,TRUE,\n-38,46,TRUE,\n-46,38,TRUE,\n-46,46,TRUE,\n-46,46,TRUE,\n-39,39,TRUE,\n-17,148,TRUE,\n-17,148,TRUE,\n-17,17,TRUE,\n-17,148,TRUE,\n-38,38,TRUE,\n-39,39,TRUE,\n-38,38,TRUE,\n-17,17,TRUE,\n-17,17,TRUE,\n-17,17,TRUE,\n-38,38,TRUE,\n-38,103,TRUE,\n-103,38,TRUE,\n-103,103,TRUE,\n-40,39,TRUE,\n-106,106,TRUE,\n-106,106,TRUE,\n-40,72,TRUE,\n-40,73,TRUE,\n-17,17,TRUE,\n-106,106,TRUE,\n-17,17,TRUE,\n-40,39,TRUE,\n-106,106,TRUE,\n-40,118,TRUE,\n-40,39,TRUE,\n-17,17,TRUE,\n-38,38,TRUE,\n-17,17,TRUE,\n-106,106,TRUE,\n-106,106,TRUE,\n-65,103,TRUE,\n-103,103,TRUE,\n-17,17,TRUE,\n-16,16,TRUE,\n-44,44,TRUE,\n-106,106,TRUE,\n-40,72,TRUE,\n-40,73,TRUE,\n-16,16,TRUE,\n-16,17,TRUE,\n-17,16,TRUE,\n-17,17,TRUE,\n-38,38,TRUE,\n-106,106,TRUE,\n-106,106,TRUE,\n-16,16,TRUE,\n-16,16,TRUE,\n-17,17,TRUE,\n-383,16,TRUE,\n-106,106,TRUE,\n-16,16,TRUE,\n-16,17,TRUE,\n-17,16,TRUE,\n-17,17,TRUE,\n-65,103,TRUE,\n671,-671,TRUE,\n671,-671,TRUE,\n671,-671,TRUE,\n-85,85,TRUE,\n-44,TRUE,\n-40,39,TRUE,\n-38,38,TRUE,\n-157,157,TRUE,\n-40,39,TRUE,\n-38,38,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-40,39,TRUE,\n-40,404,TRUE,\n-40,606,TRUE,\n-40,-44,TRUE,\n-65,103,TRUE,\n-40,39,TRUE,\n-44,44,TRUE,\n-38,38,TRUE,\n-38,38,TRUE,\n-38,38,TRUE,\n-46,46,TRUE,\n-38,38,TRUE,\n-16,16,TRUE,\n-17,17,TRUE,\n-103,103,TRUE,\n-38,38,TRUE,\n-38,38,TRUE,\n-38,38,TRUE,\n-38,38,TRUE,\n-85,85,TRUE,\n-65,38,TRUE,\n-65,103,TRUE,\n-40,39,TRUE,\n-46,46,TRUE,\n-40,39,TRUE,\n-85,85,TRUE,\n-85,85,TRUE,\n-46,46,TRUE,\n-40,39,TRUE,\n-85,85,TRUE,\n-38,38,TRUE,\n-40,39,TRUE,\n-85,85,TRUE,\n-40,72,TRUE,\n-40,73,TRUE,\n-65,103,TRUE,\n-103,103,TRUE,\n-40,39,TRUE,\n-38,38,TRUE,\n-46,46,TRUE,\n-38,38,TRUE,\n-85,85,TRUE,\n-40,39,TRUE,\n-38,38,TRUE,\n-157,157,TRUE,\n-38,38,TRUE,\n-38,85,TRUE,\n-85,38,TRUE,\n-85,85,TRUE,\n-85,85,TRUE,\n-40,39,TRUE,\n-40,3

character(0)

In [78]:
a <-"56 66 231 89 56 66"
str_split(a, " ")[[1]][2]


[1] "66"

In [53]:
write.table(dat, "/home/phi/KGMiner/input_cpp.txt", sep="\t", row.names = FALSE,col.names = FALSE)
colnames(data)


NULL

In [7]:
library(utils)
library(stringr)
  
command <- paste("hpath", dat[1,1], dat[1,2], DISCARD_REL, 3, "F", "P", sep=" ")
paths <- request(command)

In [5]:
paths


[1] "find 138 paths" "29,-29,"        "671,-671,"      "671,-671,"     
  [5] "671,-671,"      "-44,"           "-40,39,"        "-38,85,"       
  [9] "-38,38,"        "-115,16,"       "-40,404,"       "-40,606,"      
 [13] "-40,-44,"       "-38,46,"        "-46,46,"        "-40,39,"       
 [17] "-85,85,"        "-17,17,"        "-40,39,"        "-17,17,"       
 [21] "-46,46,"        "-39,240,"       "-38,103,"       "-40,39,"       
 [25] "-40,39,"        "-29,253,"       "-40,72,"        "-40,73,"       
 [29] "-115,16,"       "-15,16,"        "-15,17,"        "-38,38,"       
 [33] "-38,46,"        "-46,46,"        "-17,17,"        "-40,39,"       
 [37] "-40,39,"        "-40,39,"        "-38,38,"        "-46,46,"       
 [41] "-38,38,"        "-38,46,"        "-46,38,"        "-46,46,"       
 [45] "-65,38,"        "-65,46,"        "-46,46,"        "-38,38,"       
 [49] "-38,103,"       "-103,103,"      "-38,38,"        "-38,103,"      
 [53] "-103,38,"       "-103,103,"      "-167,46,"       "-38,103,"      
 [57] "-39,240,"       "-40,39,"        "-115,157,"      "-38,46,"       
 [61] "-38,38,"        "-40,39,"        "-46,38,"        "-46,167,"      
 [65] "-115,16,"       "-46,38,"        "-46,167,"       "-40,39,"       
 [69] "-39,240,"       "-40,39,"        "-46,46,"        "-40,39,"       
 [73] "-38,38,"        "-29,253,"       "-40,39,"        "-46,38,"       
 [77] "-40,39,"        "-39,39,"        "-39,39,"        "-39,240,"      
 [81] "-15,16,"        "-15,17,"        "-115,16,"       "-29,253,"      
 [85] "-17,17,"        "-38,46,"        "-46,46,"        "-17,17,"       
 [89] "-17,17,"        "-85,85,"        "-39,240,"       "-39,240,"      
 [93] "-38,38,"        "-85,85,"        "-38,38,"        "-38,38,"       
 [97] "-46,46,"        "-38,103,"       "-38,38,"        "-39,240,"      
[101] "-39,240,"       "-39,39,"        "-226,226,"      "-38,38,"       
[105] "-253,253,"      "-16,16,"        "-103,103,"      "-38,38,"       
[109] "-39,39,"        "-39,39,"        "-157,157,"      "-17,17,"       
[113] "-294,294,"      "-46,46,"        "-65,46,"        "-15,16,"       
[117] "-15,17,"        "-38,38,"        "-38,103,"       "-103,38,"      
[121] "-103,103,"      "-294,294,"      "-40,39,"        "-38,38,"       
[125] "-40,39,"        "-277,38,"       "-277,277,"      "-40,39,"       
[129] "-15,16,"        "-15,17,"        "-40,39,"        "-40,39,"       
[133] "-38,46,"        "-46,46,"        "-38,38,"        "-38,46,"       
[137] "-46,38,"        "-46,46,"        "-115,16,"

In [7]:
paths <- paths[-1]

In [8]:
paths <- unlist(paths[!sapply(paths, is.null)])

In [9]:
paths


[1] "29,-29,"   "671,-671," "671,-671," "671,-671," "-44,"      "-40,39,"  
  [7] "-38,85,"   "-38,38,"   "-115,16,"  "-40,404,"  "-40,606,"  "-40,-44," 
 [13] "-38,46,"   "-46,46,"   "-40,39,"   "-85,85,"   "-17,17,"   "-40,39,"  
 [19] "-17,17,"   "-46,46,"   "-39,240,"  "-38,103,"  "-40,39,"   "-40,39,"  
 [25] "-29,253,"  "-40,72,"   "-40,73,"   "-115,16,"  "-15,16,"   "-15,17,"  
 [31] "-38,38,"   "-38,46,"   "-46,46,"   "-17,17,"   "-40,39,"   "-40,39,"  
 [37] "-40,39,"   "-38,38,"   "-46,46,"   "-38,38,"   "-38,46,"   "-46,38,"  
 [43] "-46,46,"   "-65,38,"   "-65,46,"   "-46,46,"   "-38,38,"   "-38,103," 
 [49] "-103,103," "-38,38,"   "-38,103,"  "-103,38,"  "-103,103," "-167,46," 
 [55] "-38,103,"  "-39,240,"  "-40,39,"   "-115,157," "-38,46,"   "-38,38,"  
 [61] "-40,39,"   "-46,38,"   "-46,167,"  "-115,16,"  "-46,38,"   "-46,167," 
 [67] "-40,39,"   "-39,240,"  "-40,39,"   "-46,46,"   "-40,39,"   "-38,38,"  
 [73] "-29,253,"  "-40,39,"   "-46,38,"   "-40,39,"   "-39,39,"   "-39,39,"  
 [79] "-39,240,"  "-15,16,"   "-15,17,"   "-115,16,"  "-29,253,"  "-17,17,"  
 [85] "-38,46,"   "-46,46,"   "-17,17,"   "-17,17,"   "-85,85,"   "-39,240," 
 [91] "-39,240,"  "-38,38,"   "-85,85,"   "-38,38,"   "-38,38,"   "-46,46,"  
 [97] "-38,103,"  "-38,38,"   "-39,240,"  "-39,240,"  "-39,39,"   "-226,226,"
[103] "-38,38,"   "-253,253," "-16,16,"   "-103,103," "-38,38,"   "-39,39,"  
[109] "-39,39,"   "-157,157," "-17,17,"   "-294,294," "-46,46,"   "-65,46,"  
[115] "-15,16,"   "-15,17,"   "-38,38,"   "-38,103,"  "-103,38,"  "-103,103,"
[121] "-294,294," "-40,39,"   "-38,38,"   "-40,39,"   "-277,38,"  "-277,277,"
[127] "-40,39,"   "-15,16,"   "-15,17,"   "-40,39,"   "-40,39,"   "-38,46,"  
[133] "-46,46,"   "-38,38,"   "-38,46,"   "-46,38,"   "-46,46,"   "-115,16,"

In [10]:
paths <- as.data.frame(table(paths))

In [16]:
paths <- paths[order(-paths$Freq),]


In [17]:
t(paths$Freq)

20,17,12,8,7,7,6,5,5,5,⋯,1,1,1,1,1,1,1,1,1,1


In [18]:
rtn <- as.data.frame(t(paths$Freq))

In [19]:
rtn

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37
20,17,12,8,7,7,6,5,5,5,⋯,1,1,1,1,1,1,1,1,1,1


In [21]:
colnames(rtn) <- paths$paths

In [22]:
rtn

"-40,39,","-38,38,","-46,46,","-39,240,","-17,17,","-38,46,","-38,103,","-115,16,","-39,39,","-46,38,",⋯,"-277,38,","-38,85,","-40,-44,","-40,404,","-40,606,","-40,72,","-40,73,","-44,","-65,38,","29,-29,"
20,17,12,8,7,7,6,5,5,5,⋯,1,1,1,1,1,1,1,1,1,1


In [23]:
rtn <- cbind(label = as.logical(F), rtn)

In [24]:
rtn

label,"-40,39,","-38,38,","-46,46,","-39,240,","-17,17,","-38,46,","-38,103,","-115,16,","-39,39,",⋯,"-277,38,","-38,85,","-40,-44,","-40,404,","-40,606,","-40,72,","-40,73,","-44,","-65,38,","29,-29,"
FALSE,20,17,12,8,7,7,6,5,5,⋯,1,1,1,1,1,1,1,1,1,1


In [25]:
dat[1,1]

[1] 221

In [26]:
dat[1,2]

[1] 10387

In [18]:
command <- paste("same_ontology", 2489, "T")
src_ontology <- request(command)
src_ontology

[1] "Tasmania,California,Nordreisa,Arkansas,Pulaski_County,_Arkansas,Louisiana,West_Feliciana_Parish,_Louisiana,Queensland,DeSoto_Parish,_Louisiana,Klamath_County,_Oregon,Alabama,Monroe_County,_Alabama,Wisconsin,Price_County,_Wisconsin,Lincoln_County,_Wisconsin,Pennsylvania,Virginia,Dickenson_County,_Virginia,Wise_County,_Virginia,Russell_County,_Virginia,Florida,Berlin,Illinois,Collie,_Western_Australia,Western_Australia,Warwickshire,Moscow,Arkansas_County,_Arkansas,Missouri,Augusta_County,_Virginia,Texas,Michigan,Tazewell_County,_Virginia,Liguria,Buchanan_County,_Virginia,Skjervøy,Massachusetts,Wythe_County,_Virginia,Pskov_Oblast,Saint_Petersburg,Bedford_County,_Virginia,New_South_Wales,Hochsauerlandkreis,North_Rhine-Westphalia,Mie_Prefecture,Sanjak_of_Sofia,Elverum,Ohio,Meigs_County,_Ohio,Mississippi,Marshall_County,_Mississippi,Maryland,Sakarya_Province,Kars_Province,West_Papua_(region),Saitama_Prefecture,Hokkaido,Lincolnshire,Gunma_Prefecture,Yamanashi_Prefecture,Kanagawa_Prefecture,Victoria_(Australia),Kagoshima_Prefecture,Nottinghamshire,Lower_Saxony,Washington_(state),Bryan_County,_Oklahoma,Oklahoma,Laramie_County,_Wyoming,Wyoming,New_Jersey,Sanjak_of_Delvina,Georgia_(U.S._state),Clay_County,_Georgia,New_York,Dane_County,_Wisconsin,Vermont,Cortland_County,_New_York,Onondaga_County,_New_York,Worcestershire,Drač_County,Snåsa,Carbon_County,_Pennsylvania,Tennessee,Decatur_County,_Tennessee,Alexandrovsky_District,_Stavropol_Krai,Andropovsky_District,Blagodarnensky_District,Arzgirsky_District,Apanasenkovsky_District,Budyonnovsky_District,Georgiyevsky_District,Ipatovsky_District,Grachyovsky_District,_Stavropol_Krai,Izobilnensky_District,Kirovsky_District,_Stavropol_Krai,Kochubeyevsky_District,Krasnogvardeysky_District,_Stavropol_Krai,Kursky_District,_Stavropol_Krai,Levokumsky_District,Mineralovodsky_District,Neftekumsky_District,Novoalexandrovsky_District,Novoselitsky_District,Petrovsky_District,_Stavropol_Krai,Predgorny_District,Sovetsky_District,_Stavropol_Krai,Stepnovsky_District,Shpakovsky_District,Trunovsky_District,Turkmensky_District,Baleysky_District,Borzinsky_District,Chitinsky_District,Chernyshevsky_District,Gazimuro-Zavodsky_District,Kalarsky_District,Kalgansky_District,Krasnochikoysky_District,Krasnokamensky_District,Kyrinsky_District,Mogochinsky_District,Nerchinsko-Zavodsky_District,Nerchinsky_District,Olovyanninsky_District,Ononsky_District,Petrovsk-Zabaykalsky_District,Shelopuginsky_District,Priargunsky_District,Shilkinsky_District,Sretensky_District,Tungokochensky_District,Ulyotovsky_District,Zabaykalsky_District,Bureysky_District,Mazanovsky_District,Magdagachinsky_District,Oktyabrsky_District,_Amur_Oblast,Mikhaylovsky_District,_Amur_Oblast,Selemdzhinsky_District,Seryshevsky_District,Belogorsky_District,Skovorodinsky_District,Zavitinsky_District,Blagoveshchensky_District,_Amur_Oblast,Arkharinsky_District,Ivanovsky_District,_Amur_Oblast,Konstantinovsky_District,_Amur_Oblast,Romnensky_District,Shimanovsky_District,Narimanovsky_District,Tambovsky_District,_Amur_Oblast,Chernyansky_District,Zeysky_District,Belgorodsky_District,Gubkinsky_District,Tyndinsky_District,Svobodnensky_District,Novooskolsky_District,Prokhorovsky_District,Starooskolsky_District,Primorsky_District,_Arkhangelsk_Oblast,Chernoyarsky_District,Krasnoyarsky_District,_Astrakhan_Oblast,Ikryaninsky_District,Alexeyevsky_District,_Belgorod_Oblast,Kharabalinsky_District,Onezhsky_District,Kamyzyaksky_District,Borisovsky_District,Yenotayevsky_District,Ivnyansky_District,Krasnoyaruzhsky_District,Volodarsky_District,_Astrakhan_Oblast,Grayvoronsky_District,Privolzhsky_District,_Astrakhan_Oblast,Limansky_District,Rakityansky_District,Rovensky_District,_Belgorod_Oblast,Valuysky_District,Korochansky_District,Bryansky_District,Shebekinsky_District,Volokonovsky_District,Veydelevsky_District,Karachevsky_District,Yakovlevsky_District,_Belgorod_Oblast,Klimovsky_District,Brasovsky_District,Gordeyevsky_District,Dubrovsky_District,Dyatkovsky_District,Kletnyansky_Distric

In [19]:
length(src_ontology)

[1] 1

In [17]:
command <- paste("ontology", dat[300,1], "T")
src_ontology_name <- request(command)
src_ontology_name

[1] "Region,Place,Wikidata:Q532,PopulatedPlace,AdministrativeRegion,"

In [17]:
command <- paste("ontology", dat[200,1], "T")
src_ontology_name <- request(command)
src_ontology_name

[1] "Region,Place,Wikidata:Q532,PopulatedPlace,AdministrativeRegion,"

In [14]:
command <- paste("ontology", 5251, "T")
src_ontology_name <- request(command)
src_ontology_name
command <- paste("ontology", 5154, "T")
src_ontology_name <- request(command)
src_ontology_name

[1] "Place,Wikidata:Q532,Settlement,PopulatedPlace,"

[1] "Place,Wikidata:Q532,Settlement,PopulatedPlace,"

In [16]:
command <- paste("ontology", 3452, "T")
src_ontology_name <- request(command)
src_ontology_name
command <- paste("ontology", 5155, "T")
src_ontology_name <- request(command)
src_ontology_name


[1] "Country,Place,Wikidata:Q532,PopulatedPlace,"

[1] "Country,Place,Wikidata:Q532,PopulatedPlace,"

In [9]:
command <- paste("node_label", 5251)
src_ontology <- request(command)
src_ontology

[1] "Beijing"

In [11]:
src_ontology

[1] "Phoenix,_Arizona"

In [55]:
command <- paste("edge_label", 38)
src_ontology <- request(command)

In [56]:
src_ontology

[1] "birthPlace"

In [3]:
library(FSelector)
library(ggplot2)
source("experimentAPI.R")

Warning message:
“package ‘ggplot2’ was built under R version 3.4.2”Warning message:
“package ‘stringr’ was built under R version 3.4.2”

In [2]:
 install.packages("FSelector", repos="http://cran.rstudio.com/")

also installing the dependency ‘entropy’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [4]:
dat <- read.csv("/home/phi/KGMiner//gngm_celf.fullpath.csv")

In [5]:
dat

X,x
1,label
2,"10,-10,"
3,"10,54,"
4,"10,-62,"
5,"10,8,"
6,"169,-169,"
7,"169,19,"
8,"-19,-169,"
9,"-19,19,"
10,"-27,-38,"


In [6]:
colnames(dat) <- str_replace_all(str_replace(colnames(dat), "X",""), "\\.",",")

In [7]:
dat

,x
1,label
2,"10,-10,"
3,"10,54,"
4,"10,-62,"
5,"10,8,"
6,"169,-169,"
7,"169,19,"
8,"-19,-169,"
9,"-19,19,"
10,"-27,-38,"


In [8]:
weights <- information.gain(label~., dat)

ERROR: Error in terms.formula(formula, data = data): attempt to use zero-length variable name


In [9]:
data(iris)

  weights <- information.gain(Species~., iris)

In [10]:
iris

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa
4.9,3.1,1.5,0.1,setosa


In [11]:
weights

,attr_importance
Sepal.Length,0.4521286
Sepal.Width,0.2672750
Petal.Length,0.9402853
Petal.Width,0.9554360


In [13]:
features <- data.frame(importance=weights$attr_importance, path=row.names(weights))

In [15]:
features

importance,path
0.4521286,Sepal.Length
0.2672750,Sepal.Width
0.9402853,Petal.Length
0.9554360,Petal.Width


In [16]:
features <- features[order(-features$importance),]

In [17]:
features

,importance,path
4,0.9554360,Petal.Width
3,0.9402853,Petal.Length
1,0.4521286,Sepal.Length
2,0.2672750,Sepal.Width


In [1]:
library(caret)
data(GermanCredit)

Loading required package: lattice
Loading required package: ggplot2
Warning message:
“package ‘ggplot2’ was built under R version 3.4.2”

In [2]:
GermanCredit

Duration,Amount,InstallmentRatePercentage,ResidenceDuration,Age,NumberExistingCredits,NumberPeopleMaintenance,Telephone,ForeignWorker,Class,⋯,OtherInstallmentPlans.Bank,OtherInstallmentPlans.Stores,OtherInstallmentPlans.None,Housing.Rent,Housing.Own,Housing.ForFree,Job.UnemployedUnskilled,Job.UnskilledResident,Job.SkilledEmployee,Job.Management.SelfEmp.HighlyQualified
6,1169,4,4,67,2,1,0,1,Good,⋯,0,0,1,0,1,0,0,0,1,0
48,5951,2,2,22,1,1,1,1,Bad,⋯,0,0,1,0,1,0,0,0,1,0
12,2096,2,3,49,1,2,1,1,Good,⋯,0,0,1,0,1,0,0,1,0,0
42,7882,2,4,45,1,2,1,1,Good,⋯,0,0,1,0,0,1,0,0,1,0
24,4870,3,4,53,2,2,1,1,Bad,⋯,0,0,1,0,0,1,0,0,1,0
36,9055,2,4,35,1,2,0,1,Good,⋯,0,0,1,0,0,1,0,1,0,0
24,2835,3,4,53,1,1,1,1,Good,⋯,0,0,1,0,1,0,0,0,1,0
36,6948,2,2,35,1,1,0,1,Good,⋯,0,0,1,1,0,0,0,0,0,1
12,3059,2,4,61,1,1,1,1,Good,⋯,0,0,1,0,1,0,0,1,0,0
30,5234,4,2,28,2,1,1,1,Bad,⋯,0,0,1,0,1,0,0,0,0,1


In [3]:
library(RWeka)

In [5]:
model <- Logistic(Class~., GermanCredit)
res <- evaluate_Weka_classifier(model, numFolds = 10, complexity = T, class = T, seed = 233)

In [6]:
res

=== 10 Fold Cross Validation ===

=== Summary ===

Correctly Classified Instances         751               75.1    %
Incorrectly Classified Instances       249               24.9    %
Kappa statistic                          0.3769
K&B Relative Info Score              24392.7411 %
K&B Information Score                  215.1031 bits      0.2151 bits/instance
Class complexity | order 0             881.2916 bits      0.8813 bits/instance
Class complexity | scheme              768.2583 bits      0.7683 bits/instance
Complexity improvement     (Sf)        113.0333 bits      0.113  bits/instance
Mean absolute error                      0.3111
Root mean squared error                  0.4097
Relative absolute error                 74.046  %
Root relative squared error             89.3986 %
Total Number of Instances             1000     

=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0.

In [7]:
Train <- createDataPartition(GermanCredit$Class, p=0.6, list=FALSE)
training <- GermanCredit[ Train, ]
testing <- GermanCredit[ -Train, ]

In [13]:
Train

Resample1
2
3
4
5
6
8
9
10
11
12


In [10]:
length(GermanCredit)

[1] 62

In [18]:
model <- Logistic(Class~., training)
res <- evaluate_Weka_classifier(model, numFolds = 10, complexity = T, class = T, seed = 233)

In [20]:
model

Logistic Regression with ridge parameter of 1.0E-8
Coefficients...
                                            Class
Variable                                      Bad
Duration                                   0.0444
Amount                                          0
InstallmentRatePercentage                  0.3573
ResidenceDuration                          0.0785
Age                                       -0.0173
NumberExistingCredits                      0.1395
NumberPeopleMaintenance                     0.219
Telephone                                  0.4259
ForeignWorker                              1.1065
CheckingAccountStatus.lt.0                 0.8342
CheckingAccountStatus.0.to.200             0.4027
CheckingAccountStatus.gt.200              -0.2406
CheckingAccountStatus.none                -0.9644
CreditHistory.NoCredit.AllPaid             0.9804
CreditHistory.ThisBank.AllPaid             0.9657
CreditHistory.PaidDuly                     0.1168
CreditHistory.Delay              

In [28]:
pre <- predict(model, testing)

[1] Good Good Good Bad  Bad  Good Good Good Good Good Good Good Good Good Good
 [16] Good Good Good Good Good Bad  Good Good Good Good Bad  Good Bad  Good Good
 [31] Good Good Bad  Bad  Good Bad  Bad  Good Good Bad  Good Good Good Good Bad 
 [46] Good Good Bad  Good Good Good Good Good Good Bad  Good Good Bad  Bad  Good
 [61] Good Bad  Good Good Good Bad  Bad  Good Good Good Good Good Good Bad  Bad 
 [76] Good Bad  Good Good Good Bad  Bad  Good Bad  Good Good Good Good Good Good
 [91] Good Bad  Bad  Good Good Bad  Good Good Bad  Bad  Good Good Good Good Good
[106] Good Good Good Bad  Good Good Good Good Good Good Good Good Good Good Bad 
[121] Good Good Good Good Good Good Good Good Bad  Bad  Good Good Good Good Bad 
[136] Good Bad  Good Good Good Good Good Bad  Good Good Good Bad  Good Good Good
[151] Bad  Good Good Good Good Good Good Good Good Good Good Bad  Good Good Good
[166] Good Bad  Bad  Good Good Good Good Good Good Good Good Good Good Good Good
[181] Bad  Good Good Good Good Good Bad  Good Good Bad  Good Good Bad  Good Good
[196] Bad  Good Good Good Good Good Bad  Good Good Good Good Good Good Good Good
[211] Good Bad  Bad  Bad  Good Good Good Bad  Good Good Good Bad  Good Good Good
[226] Good Good Bad  Good Bad  Good Bad  Good Good Good Good Bad  Good Bad  Good
[241] Bad  Good Bad  Good Good Good Good Good Good Good Good Bad  Good Good Good
[256] Bad  Good Good Bad  Good Good Good Good Bad  Good Good Good Bad  Bad  Good
[271] Good Good Good Good Good Good Good Good Bad  Bad  Bad  Good Good Good Good
[286] Bad  Bad  Bad  Bad  Bad  Good Good Bad  Bad  Good Bad  Good Good Good Good
[301] Good Good Good Bad  Good Good Good Good Good Good Good Good Good Good Good
[316] Good Good Good Good Good Good Good Bad  Good Bad  Bad  Good Good Good Good
[331] Good Bad  Bad  Good Good Bad  Good Good Good Good Bad  Good Good Bad  Good
[346] Good Good Good Bad  Good Good Good Good Good Good Good Good Good Good Bad 
[361] Good Good Good Good Good Good Bad  Good Bad  Good Good Good Bad  Bad  Bad 
[376] Bad  Bad  Good Bad  Good Good Good Bad  Good Good Good Good Good Bad  Good
[391] Good Good Bad  Bad  Good Bad  Good Good Bad  Good
Levels: Bad Good

In [23]:
testing$Class

[1] Good Good Good Good Bad  Good Bad  Good Good Good Good Good Good Good Bad 
 [16] Good Good Good Good Good Good Good Good Bad  Good Bad  Bad  Good Bad  Good
 [31] Good Bad  Good Bad  Good Good Good Bad  Good Good Bad  Good Good Good Good
 [46] Good Bad  Bad  Good Good Good Good Bad  Good Good Good Good Bad  Good Good
 [61] Good Bad  Good Good Good Bad  Bad  Good Good Bad  Good Bad  Good Bad  Bad 
 [76] Good Good Good Good Bad  Good Good Good Good Good Good Bad  Bad  Bad  Good
 [91] Good Bad  Good Good Bad  Bad  Good Good Bad  Good Good Good Good Bad  Good
[106] Good Bad  Good Bad  Good Bad  Good Good Good Good Good Good Good Good Bad 
[121] Good Good Good Bad  Good Good Good Good Bad  Bad  Good Good Good Good Bad 
[136] Good Good Good Good Good Good Good Bad  Good Good Good Good Good Good Good
[151] Bad  Good Good Bad  Good Good Good Good Good Good Good Bad  Bad  Good Good
[166] Bad  Bad  Bad  Bad  Bad  Good Good Good Bad  Good Bad  Good Good Good Bad 
[181] Bad  Good Good Good Bad  Good Good Good Good Bad  Good Good Bad  Good Good
[196] Good Good Good Good Good Bad  Bad  Good Bad  Good Good Good Good Good Good
[211] Bad  Bad  Bad  Good Good Bad  Good Bad  Good Bad  Good Bad  Good Bad  Good
[226] Bad  Good Bad  Good Good Good Good Good Good Good Good Good Bad  Bad  Good
[241] Good Good Bad  Good Bad  Bad  Good Good Good Bad  Good Bad  Bad  Bad  Bad 
[256] Bad  Good Good Good Bad  Good Good Good Bad  Good Good Good Good Good Good
[271] Good Good Good Good Good Good Bad  Good Good Good Bad  Good Good Good Good
[286] Good Bad  Bad  Bad  Bad  Good Bad  Good Good Good Good Good Good Good Bad 
[301] Good Good Good Bad  Good Good Bad  Good Good Good Good Good Good Good Good
[316] Good Good Good Good Good Good Good Bad  Bad  Bad  Good Good Good Good Good
[331] Good Bad  Bad  Good Good Bad  Bad  Bad  Bad  Good Bad  Good Good Bad  Good
[346] Bad  Good Good Good Good Good Bad  Good Good Good Good Good Good Good Good
[361] Good Good Good Good Good Good Bad  Good Bad  Bad  Good Bad  Bad  Bad  Good
[376] Bad  Good Good Bad  Good Good Good Bad  Good Bad  Good Good Bad  Bad  Good
[391] Bad  Bad  Good Good Good Good Good Good Bad  Good
Levels: Bad Good